In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
os.environ['PRJ'] = "/Users/ymohit/skigp/"

import math
import scipy
import numpy as np
import fastmat as fm

import matplotlib.gridspec as gridspec
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore")
from pylab import rcParams
from matplotlib import pyplot as plt
from IPython.core.display import  HTML

np.random.seed(1337)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
## Loading modules from source 
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from fkigp.dataloader import DataLoader
from fkigp.configs import DatasetType, Structdict, Frameworks, GsGPType
from fkigp.gps.kernels import ScaleKernel, RBFKernel, GridInterpolationKernel
from fkigp.gps.constraints import softplus, DEFAULT_SOFTPLUS_VALUE
from fkigp.gridutils import get_basis
from fkigp.gps.gpbase import GpModel
from fkigp.gps.gpoperators import KissGpLinearOperator, GsGpLinearOperator

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Prepration and setting: 
### To demonstrate effectiveness of per-iteration of KISSGP and GSGP operators, we need K and W. 
### Also, for a fair comparison, we will consider an optimal setting (i.e. hyperparameters and number of inducing points) reported for KISSGP in Wilson et al. ICML 2015.

In [3]:
## Loading sound dataset and computing kernel matrix for sound datast

class KFunc(GpModel):
    def __init__(self, grid, dtype):
        super().__init__()
        self.covar_module = GridInterpolationKernel(
            base_kernel=ScaleKernel(RBFKernel(ard_num_dims=1)),
            grid=grid,
            dtype=dtype,
            num_dims=1
        )
            
def compute_K(train_x, grid):
    kfunc = KFunc(grid=grid, dtype=train_x.dtype)
    hypers = {
    'covar_module.base_kernel.raw_outputscale': -5.950943552288058, 
    'covar_module.base_kernel.base_kernel.raw_lengthscale': 10.895852088928223
    }
    kfunc.initialize(**hypers)
    return kfunc.covar_module._inducing_forward(is_kmm=True)
    

        
config = Structdict()
config['data_type'] = DatasetType.SOUND
data_loader=DataLoader(config=config)

train_x, train_y, test_x, test_y = data_loader.get_data()
grid =  [(1, 60000, 8000)]
W = get_basis(train_x, grid)
sigma = softplus(-10.966407775878906/2)
K_u = compute_K(train_x, grid)


## Recall KISSGP operator: WKW' + sigma^2 I. Let's compute MVM time for the same.

In [4]:

kissgp_ops = KissGpLinearOperator(W, K_u, sigma, train_x.dtype)
kissgp_result = %timeit -o kissgp_ops@np.random.rand(kissgp_ops.shape[-1])

2.64 ms ± 88.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Recall GSGP operator: KW'W + sigma^2 I. Let's compute MVM time for the same.

In [5]:
WT_times_W = fm.Sparse((W.T * W).tocsr())
WT_times_Y = W.T * train_y
YT_times_Y = train_y.T @ train_y

gsgp_ops = GsGpLinearOperator(WTW=WT_times_W, kmm=K_u, sigma=sigma, dtype=train_x.dtype)
    
    
gsgp_result = %timeit -o gsgp_ops@np.random.rand(gsgp_ops.shape[-1])

1.28 ms ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Expected speed up calculation:

In [6]:
print("GSGP took ", gsgp_result.average /  kissgp_result.average , "fraction of KISSGP!") # Expected ~ 0.5.

GSGP took  0.4847099358887178 fraction of KISSGP!
